In [1]:
from collections import deque
from dataclasses import dataclass
from typing import Hashable, Any, Optional, Dict, List

# Префіксне дерево

In [2]:
@dataclass
class Node:
    """
    Клас вузла Trie.

    Attributes:
        children (Dict[str, Node]): Діти вузла, словник символ -> вузол.
        value (Optional[Any]): Значення, якщо ключ закінчується на цьому вузлі.
    """
    children: Dict[str, "Node"]
    value: Optional[Any] = None


class Trie:
    """
    Префіксне дерево (Trie), що підтримує вставку, пошук, видалення та операції за префіксом.

    Особливості:
    - Доступ до ключів через [] та get().
    - Підтримка longest_prefix_of.
    - Пошук ключів за префіксом.
    """
    
    def __init__(self):
        """Ініціалізація Trie з порожнім кореневим вузлом та розміром 0."""
        self.root = Node(children={})
        self.size = 0

    def __setitem__(self, key: str, value: Any) -> None:
        """
        Вставка ключа зі значенням у Trie.
        
        Args:
            key (str): Ключ для вставки (непорожній рядок).
            value (Any): Значення, що зберігатиметься за ключем.
        
        Raises:
            TypeError: Якщо key порожній або не рядок.
        """
        if not key:
            raise TypeError("Key must be a non-empty string")

        current = self.root
        for char in key:
            if char not in current.children:
                current.children[char] = Node(children={})
            current = current.children[char]

        if current.value is None:
            self.size += 1
        current.value = value

    def __getitem__(self, key: str) -> Any:
        """
        Повертає значення, що відповідає ключу.

        Args:
            key (str): Ключ для пошуку.

        Returns:
            Any: Значення, що зберігається за ключем.

        Raises:
            KeyError: Якщо ключ не знайдено.
        """
        if not key:
            raise TypeError("Key must be a non-empty string")

        current = self.root
        for char in key:
            if char not in current.children:
                raise KeyError(f"Key not found: {key}")
            current = current.children[char]

        if current.value is None:
            raise KeyError(f"Key not found: {key}")
        return current.value

    def __delitem__(self, key: str) -> None:
        """
        Видаляє ключ зі значенням з Trie.

        Args:
            key (str): Ключ для видалення.

        Raises:
            TypeError: Якщо ключ порожній.
        """
        if not key:
            raise TypeError("Key must be a non-empty string")

        def _delete(node: Node, key: str, depth: int) -> bool:
            if depth == len(key):
                if node.value is not None:
                    node.value = None
                    self.size -= 1
                    return len(node.children) == 0
                return False
            char = key[depth]
            if char in node.children:
                should_delete = _delete(node.children[char], key, depth + 1)
                if should_delete:
                    del node.children[char]
                    return len(node.children) == 0 and node.value is None
            return False

        _delete(self.root, key, 0)

    def get(self, key: str, default: Any = None) -> Any:
        """
        Повертає значення ключа або default, якщо ключ не знайдено.

        Args:
            key (str): Ключ для пошуку.
            default (Any): Значення, що повертається, якщо ключ відсутній.

        Returns:
            Any: Значення за ключем або default.
        """
        try:
            return self[key]
        except KeyError:
            return default

    def __len__(self) -> int:
        """
        Повертає кількість ключів у Trie.

        Returns:
            int: Кількість ключів.
        """
        return self.size

    def keys(self) -> List[str]:
        """
        Повертає всі ключі в Trie.

        Returns:
            List[str]: Список усіх ключів.
        """
        result: List[str] = []
        self._collect(self.root, [], result)
        return result

    def keys_with_prefix(self, prefix: str) -> List[str]:
        """
        Повертає всі ключі, що починаються з префікса.

        Args:
            prefix (str): Префікс для пошуку.

        Returns:
            List[str]: Список ключів із заданим префіксом.
        """
        current = self.root
        for char in prefix:
            if char not in current.children:
                return []
            current = current.children[char]

        result: List[str] = []
        self._collect(current, list(prefix), result)
        return result

    def longest_prefix_of(self, s: str) -> str:
        """
        Знаходить найдовший префікс рядка s, що існує в Trie.

        Args:
            s (str): Рядок для пошуку префіксу.

        Returns:
            str: Найдовший префікс, що існує в Trie.
        """
        current = self.root
        longest_prefix = ""
        current_prefix = ""
        for char in s:
            if char in current.children:
                current = current.children[char]
                current_prefix += char
                if current.value is not None:
                    longest_prefix = current_prefix
            else:
                break
        return longest_prefix

    def _collect(self, node: Node, path: List[str], result: List[str]) -> None:
        """
        Приватний метод для рекурсивного обходу Trie та збору ключів.

        Args:
            node (Node): Поточний вузол.
            path (List[str]): Поточний шлях символів.
            result (List[str]): Список для збереження ключів.
        """
        if node.value is not None:
            result.append("".join(path))
        for c, n in node.children.items():
            path.append(c)
            self._collect(n, path, result)
            path.pop()


In [3]:
trie = Trie()
trie["apple"] = 1
trie["app"] = 2
trie["banana"] = 3
trie["bat"] = 4
keys = trie.keys()

keys

['app', 'apple', 'banana', 'bat']

### count_words_with_prefix

In [4]:
class CountWords(Trie):

    def count_words_with_prefix(self, prefix):
        if not isinstance(prefix, str):
            raise TypeError(
                f"Illegal argument for countWordsWithPrefix: prefix = {prefix} must be a string"
            )

        current = self.root
        for char in prefix:
            if char not in current.children:
                return 0
            current = current.children[char]

        return self._count_words(current)

    def _count_words(self, node):
        count = 1 if node.value is not None else 0
        for child in node.children.values():
            count += self._count_words(child)
        return count

In [5]:
trie = CountWords()
words = [
    "apple",
    "application",
    "appetizer",
    "banana",
    "band",
    "banner",
    "ball",
    "bat",
    "battery",
]

for index, word in enumerate(words):
    trie[word] = index

count = trie.count_words_with_prefix("app")
print(f"Кількість слів із префіксом 'app': {count}")

Кількість слів із префіксом 'app': 3


### get_corrections

In [6]:
def get_corrections(trie, word, max_distance=1):
    queue = deque([(trie.root, "", 0)])
    corrections = []

    while queue:
        current_node, current_word, current_distance = queue.popleft()

        if current_distance > max_distance:
            continue

        if current_node.value is not None and current_distance > 0:
            corrections.append(current_word)

        for char, next_node in current_node.children.items():
            next_distance = current_distance + (
                0 if char == word[len(current_word)] else 1
            )
            queue.append((next_node, current_word + char, next_distance))

    return corrections

In [ ]:
trie = Trie()
words = [
    "apple",
    "application",
    "appetizer",
    "banana",
    "band",
    "banner",
    "ball",
    "bat",
    "battery",
]

for index, word in enumerate(words):
    trie[word] = index

word_with_typo = "battary"
corrections = get_corrections(trie, word_with_typo, max_distance=1)
print(f"Можливі варіанти виправлення для '{word_with_typo}': {corrections}")

Можливі варіанти виправлення для 'battary': ['battery']
